In [13]:
import pandas as pd
import h3
from keplergl import KeplerGl


In [29]:
df_train = pd.read_csv('../00_data/train.csv')
df_test = pd.read_csv('../00_data/test.csv')

In [48]:
for h3_resolution in [8]:

    for mode in ['train', 'test']:
        
        if mode == 'train':
            df = df_train
        elif mode == 'test':
            df = df_test
        else:
            break
            
        column_pickup_id = 'pickup_h3_%s' % h3_resolution
        column_dropoff_id = 'dropoff_h3_%s' % h3_resolution

        h3_pickup_id = [h3.geo_to_h3(lat=row['pickup_latitude'],lng=row['pickup_longitude'],resolution=h3_resolution) for index, row in df.iterrows()]
        h3_dropoff_id = [h3.geo_to_h3(lat=row['dropoff_latitude'],lng=row['dropoff_longitude'],resolution=h3_resolution) for index, row in df.iterrows()]

        df[column_pickup_id] = h3_pickup_id
        df[column_dropoff_id] = h3_dropoff_id

        df.to_csv('../00_data/%s_h3.csv' % mode)

In [43]:
df_train

,Unnamed: 0,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_h3_8,dropoff_h3_8
0,0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,882a1008b3fffff,882a1008b3fffff
1,1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,882a100d2bfffff,882a100d2bfffff
2,2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,882a100d65fffff,882a100d65fffff
3,3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,882a1072c7fffff,882a1072c7fffff
4,4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,882a100883fffff,882a100883fffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458639,1458639,id2376096,2,2016-04-08 13:31:04,2016-04-08 13:44:02,4,-73.982201,40.745522,-73.994911,40.740170,N,778,882a100d29fffff,882a100d29fffff
1458640,1458640,id1049543,1,2016-01-10 07:35:15,2016-01-10 07:46:10,1,-74.000946,40.747379,-73.970184,40.796547,N,655,882a100d25fffff,882a100d25fffff
1458641,1458641,id2304944,2,2016-04-22 06:57:41,2016-04-22 07:10:25,1,-73.959129,40.768799,-74.004433,40.707371,N,764,882a100893fffff,882a100893fffff
1458642,1458642,id2714485,1,2016-01-05 15:56:26,2016-01-05 16:02:39,1,-73.982079,40.749062,-73.974632,40.757107,N,373,882a100d29fffff,882a100d29fffff


In [14]:
map_1 = KeplerGl(height=400, width=1000)
map_1.add_data(data=df_train, name='training data')
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'trainindata': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,…